In [29]:
'''
CCTV 데이터와 인구 현황 데이터를 합치고 분석하기
'''
import pandas as pd
import numpy as np

CCTV_Seoul = pd.read_csv('CCTV_in_Seoul.csv')
CCTV_Seoul.rename(columns={
        CCTV_Seoul.columns[0] : '구별'},inplace = True)
pop_Seoul = pd.read_excel('Population_in_Seoul.xls',
                          header = 2,
                          usecols = 'B,D,G,J,N'
                    )
pop_Seoul.rename(columns = {
    pop_Seoul.columns[0] : '구별',
    pop_Seoul.columns[1] : '인구수',
    pop_Seoul.columns[2] : '한국인',
    pop_Seoul.columns[3] : '외국인',
    pop_Seoul.columns[4] : '고령자'
}, inplace = True)
CCTV_Seoul['최근 증가율'] = (
        CCTV_Seoul['2014년']+
        CCTV_Seoul['2015년']+
        CCTV_Seoul['2016년']
)/ CCTV_Seoul['2013년도 이전'] * 100

pop_Seoul['외국인비율'] = pop_Seoul['외국인'] / pop_Seoul['인구수'] * 100 
pop_Seoul['고령자비율'] = pop_Seoul['고령자'] / pop_Seoul['인구수'] * 100 



print(CCTV_Seoul.head())
print(pop_Seoul.head())

'''
여기서 '구별' 을 기준으로 합치기
'''
data_result = pd.merge(CCTV_Seoul,pop_Seoul, on='구별')
data_result.head()

    구별    소계  2013년도 이전  2014년  2015년  2016년      최근 증가율
0  강남구  2780       1292    430    584    932  150.619195
1  강동구   773        379     99    155    377  166.490765
2  강북구   748        369    120    138    204  125.203252
3  강서구   884        388    258    184     81  134.793814
4  관악구  1496        846    260    390    613  149.290780
    구별         인구수        한국인       외국인        고령자     외국인비율      고령자비율
0   합계  10197604.0  9926968.0  270636.0  1321458.0  2.653918  12.958515
1  종로구    162820.0   153589.0    9231.0    25425.0  5.669451  15.615404
2   중구    133240.0   124312.0    8928.0    20764.0  6.700690  15.583909
3  용산구    244203.0   229456.0   14747.0    36231.0  6.038828  14.836427
4  성동구    311244.0   303380.0    7864.0    39997.0  2.526635  12.850689


,구별,소계,2013년도 이전,2014년,2015년,2016년,최근 증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,2780,1292,430,584,932,150.619195,570500.0,565550.0,4950.0,63167.0,0.867660,11.072217
1,강동구,773,379,99,155,377,166.490765,453233.0,449019.0,4214.0,54622.0,0.929765,12.051638
2,강북구,748,369,120,138,204,125.203252,330192.0,326686.0,3506.0,54813.0,1.061806,16.600342
3,강서구,884,388,258,184,81,134.793814,603772.0,597248.0,6524.0,72548.0,1.080540,12.015794
4,관악구,1496,846,260,390,613,149.290780,525515.0,507203.0,18312.0,68082.0,3.484582,12.955291


In [30]:
'''
이제부터는 의미 없는 컬럼을 지우도록 합시다.
행 방향으로 삭제하는 것은 drop이고 (mysql에서는 delete where 을 사용)
data_result.drop(['행 이름','행 이름'])
열 방향으로 삭제하는 명령은 del입니다. (mysql에서는 alter table '테이블명' drop '컬럼명')
'''
del data_result['2013년도 이전']
del data_result['2014년']
del data_result['2015년']
del data_result['2016년']

data_result.head()


,구별,소계,최근 증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,2780,150.619195,570500.0,565550.0,4950.0,63167.0,0.867660,11.072217
1,강동구,773,166.490765,453233.0,449019.0,4214.0,54622.0,0.929765,12.051638
2,강북구,748,125.203252,330192.0,326686.0,3506.0,54813.0,1.061806,16.600342
3,강서구,884,134.793814,603772.0,597248.0,6524.0,72548.0,1.080540,12.015794
4,관악구,1496,149.290780,525515.0,507203.0,18312.0,68082.0,3.484582,12.955291


In [31]:
data_result.rename(columns={
    data_result.columns[1]:'CCTV 개수',
    data_result.columns[2]:'CCTV 최근 증가율'
},inplace = True)
data_result.head()

,구별,CCTV 개수,CCTV 최근 증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,2780,150.619195,570500.0,565550.0,4950.0,63167.0,0.867660,11.072217
1,강동구,773,166.490765,453233.0,449019.0,4214.0,54622.0,0.929765,12.051638
2,강북구,748,125.203252,330192.0,326686.0,3506.0,54813.0,1.061806,16.600342
3,강서구,884,134.793814,603772.0,597248.0,6524.0,72548.0,1.080540,12.015794
4,관악구,1496,149.290780,525515.0,507203.0,18312.0,68082.0,3.484582,12.955291


In [32]:
'''
나중에 그래프 그릴 때를 생각하면 index를 구 이름으로 설정하는게 편합니다.
가지고 있는 열 중에 하나를 인덱스로 설정하는 명령이 set_index 입니다.
'''
data_result.set_index('구별', inplace = True)
data_result.head()

,CCTV 개수,CCTV 최근 증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
구별,,,,,,,,
강남구,2780,150.619195,570500.0,565550.0,4950.0,63167.0,0.867660,11.072217
강동구,773,166.490765,453233.0,449019.0,4214.0,54622.0,0.929765,12.051638
강북구,748,125.203252,330192.0,326686.0,3506.0,54813.0,1.061806,16.600342
강서구,884,134.793814,603772.0,597248.0,6524.0,72548.0,1.080540,12.015794
관악구,1496,149.290780,525515.0,507203.0,18312.0,68082.0,3.484582,12.955291


In [35]:
'''
대충 멋진 정리는 되었습니다. 여기서 고령자비율, 외국인비율, 인구수 중에서 무슨 데이터를 CCTV와 비교할지 정해봅시다.
Compatibility 를 볼 수 있는 가장 단순한 작업이 상관계수를 보는 것
0.1 이하면 거의 무시, 0.3 이하면 약한 상관관계, 0.7 이하면 뚜렷한 상관관계
numpy에 있는 corrcoef(Correlation coefficient) 명령을 사용하면 할 수 있습니다. 이건 근데 행렬로 다옴
대각선 제외하고 다른 값을 읽으면 됨. 이거 R 때 했지?
'''
print(np.corrcoef(data_result['고령자비율'],data_result['CCTV 개수']))
print(np.corrcoef(data_result['외국인비율'],data_result['CCTV 개수']))
np.corrcoef(data_result['인구수'],data_result['CCTV 개수'])
# 인구수와 CCTV 개수에는 약한 상관관계가 있음을 알 수 있습니다. 좀 더 알아볼까요?
'''
CCTV가 많이 설치된 구와 인구수가 많이 설치된 구를 비교하는데는 어떤 방법이 좋을까요?
시각적으로 비교해 봅시다!
'''

[[ 1.         -0.28078554]
 [-0.28078554  1.        ]]
[[ 1.         -0.13607433]
 [-0.13607433  1.        ]]


array([[1.        , 0.30634228],
       [0.30634228, 1.        ]])